Aqui neste ecodigo nós vamos realizar o back-teste de um portfolio de trade.


A ideis principal é pegar um indicador de preço/tempo qualquer, que tenha atualização diaria, e através disso realizar os seguintes passos:

Suponha um dia D
   1) Se a predição que no dia D+1 for negativa, venda as ações presentes; Se for positiva, mantenha.
   2) Quando o lucro atual superar o valor da ação, aumente a quantidade de ações.




Comecemos pegando uma ação qualquer que servirá como exemplo do indice

In [89]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable


In [90]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [98]:
ticker = "PBR"

# Puxa os dados
petrobras = yf.Ticker(ticker)

# Define o período de interesse
# 'YYYY-MM-DD'
start_date = "2000-01-01"
end_date = "2024-01-01"



tabela_periodo = petrobras.history(start=start_date, end=end_date)


df = pd.DataFrame(tabela_periodo)


df = df.reset_index()

/home/felipe/.local/lib/python3.10/site-packages/yfinance/base.py:291: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/home/felipe/.local/lib/python3.10/site-packages/yfinance/base.py:292: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [92]:
dados = df.copy()

go.Figure(data=[go.Candlestick(x=dados.index,
                open=dados['Open'],
                high=dados['High'],
                low=dados['Low'],
                close=dados['Close'])])

In [93]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
dtype: object

In [97]:
df['Data'] = df['Date'].date()

#df = df.drop(columns='Date', axis=1)



AttributeError: 'Series' object has no attribute 'date'

In [49]:





''' Vamos aqui conferir se as datas são consecutivas e se não falta nenhuma'''
df = df.sort_values('Data')

# Cria uma série de datas consecutivas entre a data mínima e máxima
data_range = pd.date_range(start=df['Data'].min(), end=df['Data'].max())

# Verifica se todas as datas do DataFrame estão presentes no range de datas
todos_consecutivos = df['Data'].isin(data_range).all()


# Mostra o resultado
if todos_consecutivos:
    print("Todas as datas são consecutivas.")
else:
    print("Existem datas não consecutivas.")

Existem datas não consecutivas.


In [51]:


# Cria uma nova coluna apenas com o dia

#df['Dia'] = df['Data'].dt.day
#df['Month'] =  df['Data'].dt.month
#df['Year'] =  df['Data'].dt.year



<div style="border-top: 2px solid red; text-align: center; padding: 10px 0;">
  <span style="color: red; font-size: 24px; font-weight: bold;">ATENÇÃO: Preciso mudar o tipo de dado do df['Date']. Devo retirar o horario e manter somente o formato data</span>
  <div style="border-top: 2px solid red; text-align: center; padding: 10px 0;">
</div>


In [82]:



class BackTest:
    def __init__(self, indice, capital_inicial,start_day, end_day):
        self.indice = indice
        self.capital_inicial = capital_inicial
        self.start_day = pd.to_datetime(start_day)
        self.end_day = pd.to_datetime(end_day)



    ''' Aqui é a função para decidir se compra, vende, ou mantém ação'''
    def CVM(self):
        # Garante que as datas estão no formato correto e coloca a coluna 'Data' como índice
        df['Data'] = pd.to_datetime(df['Data'])
        df.set_index('Data', inplace=True)  # Usa a coluna 'Data' como índice

        current_day = self.start_day

        valor_inicial_acao = df.iloc[0]['Close']
        qntdd_ac = int(self.capital_inicial / valor_inicial_acao)
        restante_caixa = self.capital_inicial - qntdd_ac * valor_inicial_acao

        tab = [self.capital_inicial]

        while current_day <= self.end_day:
            if current_day not in df.index:  # Verifica se a data existe no índice
                print("não", current_day)
                current_day += pd.DateOffset(days=1)
                continue

            valor_atual = df.loc[current_day]['Close']
            proximo_dia = current_day + pd.DateOffset(days=1)  # Adiciona 1 dia corretamente
            
            if proximo_dia not in df.index:  # Verifica se o dia seguinte existe
                break

            valor_posterior = df.loc[proximo_dia]['Close']

            if valor_posterior < valor_atual:  # Se o valor da ação cairá
                aux = qntdd_ac * valor_atual
                qntdd_ac = 0
                restante_caixa += aux

            elif valor_posterior > valor_atual:  # Se o valor da ação subirá
                compra_possivel = int(restante_caixa / valor_atual)
                if compra_possivel >= 1:
                    qntdd_ac += compra_possivel
                    restante_caixa -= compra_possivel * valor_atual

            pat_loop = qntdd_ac * valor_atual + restante_caixa
            tab.append(pat_loop)

            current_day += pd.DateOffset(days=1)

        return tab


cap_inicial = 1500
data_inicial = '2000-08-21'
data_final = '2000-10-19'
teste = BackTest(df, cap_inicial, data_inicial, data_final)
sera = teste.CVM()


KeyError: 'Data'

In [77]:
cap_inicial = 1500
data_inicial = '2000-08-21'
data_final = '2000-10-19'
teste = BackTest(df,cap_inicial, data_inicial, data_final)
sera = teste.CVM()

sera

KeyError: Timestamp('2000-08-21 00:00:00')

In [26]:
for i in range(10):
    if i ==4:
        pass
    print(i)

0
1
2
3
4
5
6
7
8
9
